# Linear Regression Diagnostics

[Resource](https://www.statsmodels.org/dev/examples/notebooks/generated/linear_regression_diagnostics_plots.html)

In real-life, relation between response and target variables are seldom linear. Here, we make use of outputs of `statsmodels` to visualize and identify potential problems that can occur from fitting a linear regression model to non-linear relations.